In [28]:
# import facenet_pytorch as face
from facenet_pytorch import MTCNN
from torchvision.transforms.functional import to_pil_image
import torch
import cv2
import numpy as np
from PIL import Image
import os

In [29]:
from dotenv import load_dotenv
load_dotenv()

register_path = os.getenv("register_path")
# path = "/mnt/c/Users/andre/OneDrive/Pictures/Facial_Recognition/"
mtcnn = MTCNN(image_size=160, margin=10)

for item in os.listdir(register_path):
    file = os.path.join(register_path, item)
    img = Image.open(file)
    aligned_face = mtcnn(img)
    aligned_pil = to_pil_image(aligned_face)
    aligned_pil.save(f"aligned_faces/{item}")



In [30]:
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
import json

model = InceptionResnetV1(pretrained="vggface2").eval()

transform = transforms.Compose([
    transforms.ToTensor()
])

In [31]:
save_path = os.getenv("save_path")
embeddings = []

for item in os.listdir(save_path):
    file = os.path.join(save_path, item)
    img = Image.open(file)
    img_tensor = transform(img)
    img_tensor = img_tensor.unsqueeze(0)
    with torch.no_grad():
        embedding = model(img_tensor)
    embedding_np = embedding.squeeze(0).numpy()
    embedding_list = embedding_np.tolist()
    embeddings.append(embedding_list)

data = {"andrew": embeddings}
with open("embeddings.json", "w") as f:
    json.dump(data, f)


In [32]:
sigin_image = os.getenv("signin_image")
img = Image.open(sigin_image)
aligned_face = mtcnn(img)
aligned_face = aligned_face.unsqueeze(0)
with torch.no_grad():
    new_embedding = model(aligned_face)
new_embedding_np = new_embedding.squeeze(0).numpy()

with open("embeddings.json", "r") as f:
    data = json.load(f)

stored_embeddings = data["andrew"]

distances = [
    np.linalg.norm(new_embedding_np - np.array(emb))
    for emb in stored_embeddings
]
min_distance = min(distances)

In [33]:
threshold = 0.5

if min_distance < threshold:
    print(f"✅ Sign-in successful. Matched with 'andrew' at distance {min_distance:.4f}.")
else:
    print(f"❌ Sign-in failed. Closest distance {min_distance:.4f} exceeds threshold.")

✅ Sign-in successful. Matched with 'andrew' at distance 0.4633.
